In [1]:
import sys
import pandas as pd
import json
import numpy as np
import torch
from torch import Tensor
import pickle
from tqdm.auto import tqdm

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard, calc_jaccard_mean

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-15 17:20:29,069 log.py               31   [INFO] [__init__] skip wandb init 


## optuna ensemble

In [3]:
import pickle

with open("../data/dataset/optuna_ens/prep_dfs.pkl", "rb") as fin:
    prep_dfs = pickle.load(fin)

In [4]:
for k in prep_dfs:
    prep_dfs[k] = prep_dfs[k][prep_dfs[k].start_logits.notnull()]

In [5]:
optuna_exp_ids = ["e048", "e049", "e051", "e053", "e054", "e073", "e074"]

In [6]:
from src.postprocessor.factory import PostprocessorFactory

postprocessor = PostprocessorFactory(
    **{
        "postprocessor_type": "baseline_kernel",
        "n_best_size": 50,
        "max_answer_length": 30,
        "use_chars_length": False,
        "text_postprocess": "mypospro_ver1",
        "use_multiprocess": True,
    },
    logger=logger
).create()

In [7]:
def objective(trial):
    samples = [1./len(optuna_exp_ids)]*len(optuna_exp_ids)
    for i in range(len(optuna_exp_ids)):
        samples[i] = trial.suggest_uniform(optuna_exp_ids[i], 0, 1)
    
    w = [samples[i]/sum(samples) for i in range(len(samples))]
    
    ids = prep_dfs[optuna_exp_ids[0]]["id"]
    contexts = prep_dfs[optuna_exp_ids[0]]["context"]
    answer_texts = prep_dfs[optuna_exp_ids[0]]["answer_text"]
    offset_mappings = prep_dfs[optuna_exp_ids[0]]["offset_mapping"]
    
    start_logits = w[0] * prep_dfs[optuna_exp_ids[0]]["start_logits"]
    end_logits = w[0] * prep_dfs[optuna_exp_ids[0]]["end_logits"]
    for i in range(1, len(optuna_exp_ids)):
        start_logits += w[i] * prep_dfs[optuna_exp_ids[i]]["start_logits"]
        end_logits += w[i] * prep_dfs[optuna_exp_ids[i]]["end_logits"]

    pospro_ids, pospro_answer_texts, pospro_answer_preds = postprocessor(
        ids=ids,
        contexts=contexts,
        answer_texts=answer_texts,
        offset_mappings=offset_mappings,
        start_logits=start_logits,
        end_logits=end_logits,
    )
    val_jaccard = calc_jaccard_mean(
        text_trues=pospro_answer_texts, text_preds=pospro_answer_preds
    )
    return val_jaccard

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-11-15 17:20:59,744] A new study created in memory with name: no-name-2cf9ba04-12f2-4ff6-be6a-5798ddfe075c
2021-11-15 17:21:00,792 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:21:07,638] Trial 0 finished with value: 0.7177803756686166 and parameters: {'e048': 0.938660232344256, 'e049': 0.2704948954722023, 'e051': 0.014850868289734143, 'e053': 0.1905745051280272, 'e054': 0.05489197235059218, 'e073': 0.4333540782964327, 'e074': 0.9235486602538849}. Best is trial 0 with value: 0.7177803756686166.
2021-11-15 17:21:08,626 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:21:15,593] Trial 1 finished with value: 0.716782029809409 and parameters: {'e048': 0.49024049506639855, 'e049': 0.554188711431312, 'e051': 0.46751614538933695, 'e053': 0.4634478330621994, 'e054': 0.6628962939206945, 'e073': 0.2904774779832594, 'e074': 0.1952091936191912}. Best is trial 0 with value: 0.7177803756686166.
2021-11-15 17:21:16,597 postprocessor.py     58   [INFO] [__call__] start postprocessing 


...

  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:32:08,336] Trial 87 finished with value: 0.7290362402907289 and parameters: {'e048': 0.2399962564250631, 'e049': 0.07020220227096292, 'e051': 0.6217465707291432, 'e053': 0.9517404189086505, 'e054': 0.7026578809486754, 'e073': 0.9120345272365062, 'e074': 0.9816350323809153}. Best is trial 87 with value: 0.7290362402907289.
2021-11-15 17:32:09,302 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

[I 2021-11-15 17:32:16,976] Trial 88 finished with value: 0.7285980937574298 and parameters: {'e048': 0.2454007687893002, 'e049': 0.07695769600417644, 'e051': 0.7000440838467737, 'e053': 0.9433036154699009, 'e054': 0.7154245697016095, 'e073': 0.9808669285053095, 'e074': 0.9795022461130721}. Best is trial 87 with value: 0.7290362402907289.
2021-11-15 17:32:18,005 postprocessor.py     58   [INFO] [__call__] start postprocessing 


  0%|          | 0/1114 [00:00<?, ?it/s]

In [ ]:
d=study.best_params 
v=study.best_value
print(v, d)